In [42]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("local") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 15:33:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Q1
What is the version of Spark in use?

In [3]:
spark.version

'3.3.0'

In [4]:
path = r'../data/fhvhv_tripdata_2021-06.csv.gz'

In [55]:
pdf = pd.read_csv(path, compression="gzip", nrows=100)

In [56]:
pdf.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [19]:
pdf.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [21]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [22]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(path)

In [23]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/03/04 16:00:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers


### Q2

Repartition it to 12 partitions and save it to parquet.
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? 

In [25]:
df = df.repartition(12)

In [32]:
df.write \
.mode("overwrite") \
.parquet('../data/pq/fhvhv/2021/06/')

23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/03/04 16:04:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers


23/03/04 16:04:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
23/03/04 16:04:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/03/04 16:04:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/03/04 16:04:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/04 16:04:44 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [41]:
!ls -lh "../data/pq/fhvhv/2021/06"

total 591360
-rw-r--r--  1 home  staff     0B Mar  4 16:04 _SUCCESS
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00000-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00001-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00002-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00003-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00004-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00005-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00006-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  staff    24M Mar  4 16:04 part-00007-e8c252af-323e-4185-b32e-90a9e820cea4-c000.snappy.parquet
-rw-r--r--  1 home  

### Q3

How many taxi trips were there on June 15?

Consider only trips that started on June 15.

In [43]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [45]:
df.createOrReplaceTempView('fhvhv_2021_06')

In [46]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



### Q4

Now calculate the duration for each trip.
How long was the longest trip in Hours?

In [49]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
    .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
+-----------+-------------+



In [50]:
df.createOrReplaceTempView('fhvhv_max_duration')

In [54]:
spark.sql("""
SELECT
    to_date(pickup_datetime) pickup_date,
    max((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) max_duration
FROM 
    fhvhv_max_duration
GROUP BY
    pickup_date
ORDER BY
    max_duration desc
LIMIT 5;
""").show()

+-----------+------------------+
|pickup_date|      max_duration|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
+-----------+------------------+



### Q6

What is the name of the most frequent pickup location zone?

In [68]:
df_zones = spark.read.csv('../data/taxi_zone_lookup.csv', header="true")

In [69]:
df_zones.head(5)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [70]:
df_zones.createOrReplaceTempView('zones')

In [71]:
spark.sql("""
SELECT
    *
FROM 
    zones
LIMIT 2;
""").show()

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
|         2| Queens|   Jamaica Bay|   Boro Zone|
+----------+-------+--------------+------------+



In [ ]:
spark.sql("""
SELECT
    Zone,
    count(*) as max_zones
FROM 
    fhvhv_2021_06 b left join zones a  on a.LocationID = b.PULocationID
GROUP BY
    Zone
ORDER BY
    max_zones desc
LIMIT 2;
""").show()